In [1]:
def adf_training(variance, D, f_debug, f_step_validation, f_validation, c_ds, c_fi):
    theta_t_m = np.array([0.] * (D)) # mean of thetas at t
    theta_t_v = np.array([variance] * (D)) # variance of thetas at t
    n = np.array([0.] * D)

    start_time = time.time()
    
    log_loss_adf_training = 0.
    arr_log_loss_adf_test = []
    
    f = open(c_fi.file_path)
    fn = c_fi.l_header_names
    
    for t, row in enumerate(DictReader(f, fieldnames=fn, delimiter=c_fi.seperator)):
    
        if len(c_fi.l_skip_columns) > 0 :
            for i in range(len(c_fi.l_skip_columns)):
                del row[(c_fi.l_skip_columns)[i]] # for titanic
        
        if t == 0 & c_fi.f_having_header:
            continue
            
        if t < c_ds.num_train_data_start:
            continue
        # Start of ...


        y = 1. if row[c_fi.ylab] == '1' else 0.
        del row[c_fi.ylab]
        
        x = get_x_mmh3(row, D)

        # Predictive distribution for s_t ~ N(s_t_m_old, s_t_v_old)
        s_t_m_old = sum(theta_t_m[x])
        s_t_v_old = sum(theta_t_v[x])

        # Posterior distribution for s_t
        s_t_m, s_t_v = get_s_t_new(y, s_t_m_old, s_t_v_old)

        # Changes in s_t
        delta_m = s_t_m - s_t_m_old
        delta_v = s_t_v - s_t_v_old

        # Updating theta
        update_theta_cat(x, theta_t_m, theta_t_v, delta_m, delta_v, t, n)

        p = get_p_cat(x, theta_t_m)

        log_loss_adf_training += logloss(p, y)
        
        if f_debug:
            if y == 1.:
                print('%s\tencountered: %d\t y=%d: %f, loss:%f' % (
                    datetime.now(), (t), y, p, log_loss_adf_training/t))
            if t % c_ds.num_status_check_point == 0 and t > 1:
                print('%s\tencountered: %d\t y=%d: %f, loss:%f' % (
                    datetime.now(), (t), y, p, log_loss_adf_training/t))

        if f_step_validation:
            if t % c_ds.num_metric_check_point == 0 and t > 1:
                rt_log_loss, arr_y, arr_p = get_validation_metrics(
                        c_fi
                        , c_ds.num_test_data_start
                        , c_ds.num_test_data_end
                        , D
                        , theta_t_m
                        , f_debug)
                
                arr_log_loss_adf_test.append(rt_log_loss / (c_ds.num_test_data_end - c_ds.num_test_data_start))

        # End of ...
        if t >= c_ds.num_train_data_end:
            break
    f.close()
    
    if f_debug:
        print("---Total execution time: %s seconds ---" % (time.time() - start_time))
    
    if f_step_validation:
        return(arr_log_loss_adf_test)
    
    if f_validation:
        rt_log_loss_adf_training = log_loss_adf_training / (c_ds.num_train_data_end - c_ds.num_train_data_start)
        
        rt_log_loss_adf_test, arr_y, arr_p = get_validation_metrics(
                        c_fi
                        , c_ds.num_test_data_start
                        , c_ds.num_test_data_end
                        , D
                        , theta_t_m
                        , f_debug)
        rt_log_loss_adf_test = rt_log_loss_adf_test / (c_ds.num_test_data_end - c_ds.num_test_data_start)
        
        return((theta_t_m, rt_log_loss_adf_training, rt_log_loss_adf_test))


    